# Gathering labels

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
# %load ../common_.py
import sys
sys.path.append("../..")
from common import *

In [16]:
import pyarrow

## Load data

In [17]:
try:
    image_path = snakemake.input.image_path
    moa_path = snakemake.input.moa_path
    output = snakemake.output[0]
except NameError:
    data_root = Path("/data/gent/vo/000/gvo00070/vsc42015/datasets/BBBC021")
    image_path = data_root / "BBBC021_v1_image.csv"
    moa_path = data_root / "BBBC021_v1_moa.csv"
    output = data_root / "labels.parquet"

## meta

In [18]:
image = pandas.read_csv(image_path)
image = image.drop(columns=["Image_PathName_DAPI", "Image_PathName_Tubulin", "Image_PathName_Actin", "Image_FileName_Tubulin", "Image_FileName_Actin"])
image.columns = ["meta_" + col.lower() for col in image.columns]

In [19]:
image.head()

,meta_tablenumber,meta_imagenumber,meta_image_filename_dapi,meta_image_metadata_plate_dapi,meta_image_metadata_well_dapi,meta_replicate,meta_image_metadata_compound,meta_image_metadata_concentration
0,4,233,G10_s1_w1BEDC2073-A983-4B98-95E9-84466707A25D.tif,Week4_27481,G10,1,5-fluorouracil,0.003
1,4,234,G10_s2_w11C3B9BCC-E48F-4C2F-9D31-8F46D8B5B972.tif,Week4_27481,G10,1,5-fluorouracil,0.003
2,4,235,G10_s3_w1F4FCE330-C71C-4CA3-9815-EAF9B9876EB5.tif,Week4_27481,G10,1,5-fluorouracil,0.003
3,4,236,G10_s4_w1747818B4-FFA7-40EE-B0A0-6A5974AF2644.tif,Week4_27481,G10,1,5-fluorouracil,0.003
4,4,473,G10_s1_w10034568D-CC12-43C3-93A9-DC3782099DD3.tif,Week4_27521,G10,2,5-fluorouracil,0.003


In [23]:
image[((image["meta_image_metadata_compound"] == "ALLN") & (image["meta_image_metadata_concentration"] == 3.0))]

,meta_tablenumber,meta_imagenumber,meta_image_filename_dapi,meta_image_metadata_plate_dapi,meta_image_metadata_well_dapi,meta_replicate,meta_image_metadata_compound,meta_image_metadata_concentration
11700,2,97,Week2_180607_D06_s1_w1462FED99-AACD-4BCA-864B-...,Week2_24121,D06,1,ALLN,3.0
11701,2,98,Week2_180607_D06_s2_w19167AEA1-6B3B-4E1D-9B49-...,Week2_24121,D06,1,ALLN,3.0
11702,2,99,Week2_180607_D06_s3_w146C32A66-CB47-46C8-BD4E-...,Week2_24121,D06,1,ALLN,3.0
11703,2,100,Week2_180607_D06_s4_w19B6EE5DF-0555-458C-B768-...,Week2_24121,D06,1,ALLN,3.0
11704,2,337,Week2_180607_D06_s1_w1458598FE-07DB-4C8F-B50E-...,Week2_24141,D06,2,ALLN,3.0
11705,2,338,Week2_180607_D06_s2_w132576D7B-4923-4CD9-B3B3-...,Week2_24141,D06,2,ALLN,3.0
11706,2,339,Week2_180607_D06_s3_w1D4BDE5E3-E9FF-4D3B-8786-...,Week2_24141,D06,2,ALLN,3.0
11707,2,340,Week2_180607_D06_s4_w182246DD2-7557-488D-96C1-...,Week2_24141,D06,2,ALLN,3.0
11708,2,577,Week2_180607_D06_s1_w164F92143-D223-4926-A38D-...,Week2_24161,D06,3,ALLN,3.0
11709,2,578,Week2_180607_D06_s2_w15D22BD4F-6C40-4C79-8B44-...,Week2_24161,D06,3,ALLN,3.0


In [ ]:
moa = pandas.read_csv(moa_path)

In [ ]:
labels = image.merge(moa, left_on=("meta_image_metadata_compound", "meta_image_metadata_concentration"), right_on=("compound", "concentration"), how="inner")
labels = labels.drop(columns=["meta_image_metadata_compound", "meta_image_metadata_concentration"])

In [ ]:
labels.columns = [c if c.startswith("meta_") else "meta_"+c for c in labels.columns]

In [ ]:
pq.write_table(pyarrow.Table.from_pandas(labels), output)

In [ ]:
labels["meta_moa"].value_counts()

Microtubule stabilizers      1428
DMSO                         1320
Microtubule destabilizers     168
Aurora kinase inhibitors      144
Eg5 inhibitors                144
DNA damage                    108
DNA replication                96
Protein synthesis              96
Epithelial                     88
Protein degradation            84
Cholesterol-lowering           72
Actin disruptors               60
Kinase inhibitors              40
Name: meta_moa, dtype: int64

In [ ]:
sorted(labels[labels["meta_moa"] == "Protein degradation"]["meta_image_metadata_plate_dapi"].unique())

['Week2_24121',
 'Week2_24141',
 'Week2_24161',
 'Week2_24361',
 'Week2_24381',
 'Week2_24401',
 'Week6_31641',
 'Week6_31661',
 'Week6_31681',
 'Week7_34341',
 'Week7_34381',
 'Week7_34641']